In [1]:
from prep_data import PrepData
from anchor_boxes import *
from yolo_model import *

import numpy as np
import tensorflow as tf

In [2]:
input_shape = (676, 380, 3)
grid_dim = 13
anchor_sizes = [(0.2, 0.1), (0.3, 0.2), (0.4, 0.3), (0.5, 0.4)]
num_anchors = len(anchor_sizes)

In [3]:
p = PrepData()
train, test = p.load_images()
labels = p.connect_labels(train, grid_dim, anchor_sizes, iou_vectorized)
train, test = p.preprocess_images(train), p.preprocess_images(test)

# swap axes of x and y of the image
train = np.swapaxes(train, 1, 2)
test = np.swapaxes(test, 1, 2)
train.shape

(1001, 676, 380, 3)

In [4]:
def custom_iou_loss(y_true, y_pred):
    # Compute IOUs
    ious = iou_vectorized(y_true[..., :4], y_pred[..., :4])
    iou_loss = tf.abs(ious - y_true[..., -1]) # IOU losses - the confidence scores
    
    # Mean squared error over IOU losses
    return tf.reduce_mean(tf.square(iou_loss))

In [5]:
model = yolo_model(input_shape, grid_dim, num_anchors)
model.compile(optimizer='adam', loss=custom_iou_loss, metrics=['accuracy'])
model.fit(train, labels, epochs=3, batch_size=16, verbose=1)
model.evaluate(test, labels)

/Users/jadenvanrijswijk/Car Pose Estimation/.venv/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/3


TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got Ellipsis